In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import defaultdict, Counter
import _pickle as pickle
import random
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc

In [2]:
demographic_matrix = pickle.load(open('data/demographic_matrix.p', 'rb'))
demographic_variables = pickle.load(open('data/demographic_variables.p', 'rb'))
datetime_matrix = pickle.load(open('data/datetime_matrix.p', 'rb'))
datetime_variables = pickle.load(open('data/datetime_variables.p', 'rb'))
diag_matrix = pickle.load(open('data/diag_matrix.p', 'rb'))
diag_variables = pickle.load(open('data/diag_variables.p', 'rb'))
negative_training_set = pickle.load(open('data/negative_training_set.p', 'rb'))
positive_training_set = pickle.load(open('data/positive_training_set.p', 'rb'))

In [3]:
full_matrix = []

for pat_enc_csn_id in positive_training_set:
    to_add = demographic_matrix[pat_enc_csn_id]
    to_add += datetime_matrix[pat_enc_csn_id]
    to_add += diag_matrix[pat_enc_csn_id]
    full_matrix.append(to_add)
    
for pat_enc_csn_id in negative_training_set:
    to_add = demographic_matrix[pat_enc_csn_id]
    to_add += datetime_matrix[pat_enc_csn_id]
    to_add += diag_matrix[pat_enc_csn_id]
    full_matrix.append(to_add)

In [4]:
Counter([len(i) for i in full_matrix])

Counter({1637: 9340})

In [5]:
full_variable_list = demographic_variables + datetime_variables[:-1] + diag_variables
len(full_variable_list)

1637

In [6]:
len(positive_training_set), len(negative_training_set)

(4670, 4670)

In [7]:
covid_non_covid_visits = [1 for i in range(4670)] + [0 for i in range(4670)]
len(covid_non_covid_visits), Counter(covid_non_covid_visits)

(9340, Counter({1: 4670, 0: 4670}))

In [8]:
full_matrix = np.array(full_matrix)

In [9]:
negative_testing_set = pickle.load(open('data/negative_eval_set.p', 'rb'))
positive_testing_set = pickle.load(open('data/positive_eval_set.p', 'rb'))

In [10]:
full_matrix_testing = []

for pat_enc_csn_id in positive_testing_set:
    to_add = demographic_matrix[pat_enc_csn_id]
    to_add += datetime_matrix[pat_enc_csn_id]
    to_add += diag_matrix[pat_enc_csn_id]
    full_matrix_testing.append(to_add)
    
for pat_enc_csn_id in negative_testing_set:
    to_add = demographic_matrix[pat_enc_csn_id]
    to_add += datetime_matrix[pat_enc_csn_id]
    to_add += diag_matrix[pat_enc_csn_id]
    full_matrix_testing.append(to_add)

In [11]:
covid_non_covid_visits_testing = [1 for i in range(4670)] + [0 for i in range(4670)]
len(covid_non_covid_visits_testing), Counter(covid_non_covid_visits_testing)

(9340, Counter({1: 4670, 0: 4670}))

In [12]:
full_matrix_testing = np.array(full_matrix_testing)

In [13]:
rf = RandomForestClassifier(n_estimators=190, random_state=1, oob_score=True, max_depth=69)
rf.fit(full_matrix, covid_non_covid_visits)
estimate_covid_oob = rf.oob_decision_function_[:,1]
estimate_covid = rf.predict_proba(full_matrix)[:,1]
testing_covid = rf.predict_proba(full_matrix_testing)[:,1]
false_positive_rate, true_positive_rate, thresholds = roc_curve(covid_non_covid_visits, estimate_covid_oob)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('training_oob_auroc',roc_auc)
false_positive_rate, true_positive_rate, thresholds = roc_curve(covid_non_covid_visits, estimate_covid)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('training_auroc',roc_auc)
false_positive_rate, true_positive_rate, thresholds = roc_curve(covid_non_covid_visits_testing, testing_covid)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('testing_auroc',roc_auc)
    

training_oob_auroc 0.8957386892507186
training_auroc 0.9866616381385582
testing_auroc 0.8958289276396334


In [14]:
pickle.dump(rf, open('random_forest_model_depth_69_trees_190.p', 'wb'))

In [15]:
importances = list(rf.feature_importances_)

In [16]:
len(full_variable_list), len(importances)

(1637, 1637)

In [17]:
importance_table = pd.DataFrame(np.vstack((np.array(full_variable_list), np.array(importances))).T, columns=['feature', 'importance'])

In [18]:
importance_table.sort_values(by=['importance'], ascending=False)

,feature,importance
564,R06,0.064984
310,R09,0.062809
13,2020-04-01,0.054332
416,R05,0.025933
520,J12,0.023595
...,...,...
587,F23,0.0
1159,V01,0.0
586,D66,0.0
582,A57,0.0


In [19]:
importance_table.to_csv('data/final_feature_importances.csv', index=False)